In [2]:
import numpy as np
from pandas import DataFrame
import pandas as pd

df3 = pd.read_csv('drive/MyDrive/지하철/서울교통공사_2023년.csv', encoding='cp949')
station = dict() #역명을 숫자로 교체 (서울역 = 1 , 용산 = 2 ...)
sdx = 0
station_list = [] #역명 담을 리스트
subway_list = [] #지하철 호선 담을 리스트
week_list = [] #요일 담을 리스트(월요일 = 0 , 화요일 = 1 , 수요일 = 2 ...)


#------------------------ 2023년--------------------
#2023년 1월 1일은 일요일 -> 6
prev = "2023-01-01"
idx = 6
cnt = 0
for day , subway , st in zip(df3['수송일자'] ,df3['호선'] ,df3['역명']):
  if cnt % 2 == 0: #승차 , 하차 중 1번만 저장하기 위해서 조건문으로 해결함.
    if prev == day: #날짜가 바뀌지 않았을 때
      week_list.append(idx)
    else: #날짜가 바뀌었을 때
      idx = (idx + 1) % 7 #날짜가 증가하였기 때문에 요일도 증가
      week_list.append(idx)
      prev = day
    subway_list.append(int(subway[0])) #인덱스 0번을 숫자로 변형
    if st in station: #역명이 이전에 존재했던 경우
      station_list.append(station[st])
    else: #역명이 이전에 존재하지 않았을 경우 숫자로 변형
      station[st] = sdx
      station_list.append(sdx)
      sdx += 1
  cnt += 1


#------------------------ 2022년--------------------
df2 = pd.read_csv('drive/MyDrive/지하철/서울교통공사_2022년.csv', encoding='cp949')
#2022년 1월 1일은 토요일 -> 5
prev = "2022-01-01"
idx = 5
cnt = 0
for day , subway , st in zip(df2['수송일자'] ,df2['호선'] ,df2['역명']):
  if cnt % 2 == 0:
    if prev == day:
      week_list.append(idx)
    else:
      idx = (idx + 1) % 7
      week_list.append(idx)
      prev = day
    subway_list.append(subway)
    if st in station:
      station_list.append(station[st])
    else:
      station[st] = sdx
      station_list.append(sdx)
      sdx += 1
  cnt += 1


#------------------------ 2021년--------------------
df1 = pd.read_csv('drive/MyDrive/지하철/서울교통공사_2021년.csv', encoding='cp949')
#2022년 1월 1일은 금요일 -> 4
prev = "2021-01-01"
idx = 4
cnt = 0
for day , subway , st in zip(df1['날짜'] ,df1['호선'] ,df1['역명']):
  if cnt % 2 == 0:
    if prev == day:
      week_list.append(idx)
    else:
      idx = (idx + 1) % 7
      week_list.append(idx)
      prev = day
    subway_list.append(subway)
    if st in station:
      station_list.append(station[st])
    else:
      station[st] = sdx
      station_list.append(sdx)
      sdx += 1
  cnt += 1


X = dict()
X['역명'] = station_list
X['호선'] = subway_list
X['요일'] = week_list
X = DataFrame(X)




<ipython-input-2-a1966761d3a8>:37: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('drive/MyDrive/지하철/서울교통공사_2022년.csv', encoding='cp949')


In [3]:
X

,역명,호선,요일
0,0,1,6
1,1,1,6
2,2,1,6
3,3,1,6
4,4,1,6
...,...,...,...
234364,234,8,4
234365,235,8,4
234366,236,8,4
234367,237,8,4


In [7]:
def y_train_select(time):
  t23 = df3[time]
  t22 = df2[time]
  t21 = df1[time]
  t = list(t23) + list(t22) + list(t21)
  array = []
  cnt = 0
  for a in t:
    if cnt % 2 == 0: #승차와 하차 인원수를 합쳐주기 위해서 조건문으로 해결함.
      array.append(a)
    else:
      array[-1] += a
    cnt += 1
  y = dict()
  y['인원수'] = np.log(np.array(array) + 1) #특이값 노이즈를 줄이기 위해서 자연로그를 사용함.
  return y


times = ['06-07시간대' , '07-08시간대' , '08-09시간대' , '17-18시간대' , '18-19시간대' , '19-20시간대' ]

peoples = y_train_select(times[4])
y = DataFrame(peoples) #18-19시간대

In [8]:
y

,인원수
0,8.407378
1,7.273786
2,7.515889
3,7.336286
4,7.180831
...,...
234364,7.400010
234365,6.635947
234366,6.620073
234367,6.549651


In [10]:
import torch

stations = torch.LongTensor(station_list)
subways = torch.LongTensor(subway_list)
weeks = torch.LongTensor(week_list)
peoples = torch.FloatTensor(peoples['인원수'])

In [15]:
n_ratings = len(peoples) #데이터의 수
train_size = int(n_ratings*0.8) #train데이터와 test데이터 나누기
shuffled = torch.randperm(n_ratings) #train 데이터와 test 데이터를 나누기 전에 랜덤으로 섞어주기
#train data set
stations_train = stations[shuffled[:train_size]]
subways_train = subways[shuffled[:train_size]]
weeks_train = weeks[shuffled[:train_size]]
peoples_train = peoples[shuffled[:train_size]]
#test data set
stations_test = stations[shuffled[train_size:]]
subways_test = subways[shuffled[train_size:]]
weeks_test = weeks[shuffled[train_size:]]
peoples_test = peoples[shuffled[train_size:]]


In [26]:
n_stations = max(stations) + 1 #역명 크기
n_subways = max(subways) + 1 #호선 크기
n_weeks = max(weeks) + 1 #요일 크기
mean = peoples_train.mean()
b_st = torch.zeros(n_stations , requires_grad = True) #역명 bias 초기화
b_su = torch.zeros(n_subways , requires_grad = True) #호선 bias 초기화
b_we = torch.zeros(n_weeks , requires_grad = True) #요일 bias 초기화


optim = torch.optim.Adam([b_st , b_su , b_we] , lr = 0.1 , weight_decay = 0.00006) #업데이트 될 bias 등록 , 학습률 = 0.1 ,정규화 = 0.00006
mse = torch.nn.MSELoss() #MSE
for epoch in range(1000):
  h = mean + b_st[stations_train] + b_su[subways_train] + b_we[weeks_train] # 예측값 = 평균 + 역명_bias + 호선_bias + 요일_bias
  cost = mse(h , peoples_train) #MSE
  optim.zero_grad()
  cost.backward() #backward
  optim.step() #update
  with torch.no_grad(): #업데이트 중지
    h_train = mean + b_st[stations_train] + b_su[subways_train] + b_we[weeks_train]
    cost_train = mse(h_train , peoples_train) #train_mse
    h_test = mean + b_st[stations_test] + b_su[subways_test] + b_we[weeks_test]
    cost_test = mse(h_test , peoples_test) #test_mse
    if epoch % 100 == 0:
      print("train_mse :", cost_train.item() , end=" ")
      print("test_mse :", cost_test.item())

train_mse : 0.8257220983505249 test_mse : 0.8384049534797668
train_mse : 0.22423243522644043 test_mse : 0.23690663278102875
train_mse : 0.22419168055057526 test_mse : 0.23693367838859558
train_mse : 0.22418685257434845 test_mse : 0.23692455887794495
train_mse : 0.22418466210365295 test_mse : 0.23692017793655396
train_mse : 0.22418375313282013 test_mse : 0.2369190752506256
train_mse : 0.22418351471424103 test_mse : 0.2369179129600525
train_mse : 0.22418348491191864 test_mse : 0.2369171380996704
train_mse : 0.2242153137922287 test_mse : 0.23690836131572723
train_mse : 0.22418440878391266 test_mse : 0.236913800239563
train_mse : 0.2241833508014679 test_mse : 0.23691828548908234


In [32]:
days = ['월요일' , '화요일' , '수요일' , '목요일' , '금요일' , '토요일' , '일요일']
for i in range(7): #서울역 1호선
  temp = mean + b_st[0] + b_su[1] + b_we[i]
  result = int(np.exp(temp.item()) - 1)
  print(f"{days[i]} 서울역 1호선은 18-19시간대에 {result} 명의 사람들이 있을 예정입니다. ")

월요일 서울역 1호선은 18-19시간대에 5312 명의 사람들이 있을 예정입니다. 
화요일 서울역 1호선은 18-19시간대에 5776 명의 사람들이 있을 예정입니다. 
수요일 서울역 1호선은 18-19시간대에 5763 명의 사람들이 있을 예정입니다. 
목요일 서울역 1호선은 18-19시간대에 5905 명의 사람들이 있을 예정입니다. 
금요일 서울역 1호선은 18-19시간대에 5909 명의 사람들이 있을 예정입니다. 
토요일 서울역 1호선은 18-19시간대에 2380 명의 사람들이 있을 예정입니다. 
일요일 서울역 1호선은 18-19시간대에 1787 명의 사람들이 있을 예정입니다. 


In [33]:
for i in range(7): # 시청역 2호선
  temp = mean + b_st[1] + b_su[2] + b_we[i]
  result = int(np.exp(temp.item()) - 1)
  print(f"{days[i]} 시청 2호선은 18-19시간대에 {result} 명의 사람들이 있을 예정입니다. ")

월요일 시청 2호선은 18-19시간대에 4650 명의 사람들이 있을 예정입니다. 
화요일 시청 2호선은 18-19시간대에 5057 명의 사람들이 있을 예정입니다. 
수요일 시청 2호선은 18-19시간대에 5045 명의 사람들이 있을 예정입니다. 
목요일 시청 2호선은 18-19시간대에 5170 명의 사람들이 있을 예정입니다. 
금요일 시청 2호선은 18-19시간대에 5173 명의 사람들이 있을 예정입니다. 
토요일 시청 2호선은 18-19시간대에 2083 명의 사람들이 있을 예정입니다. 
일요일 시청 2호선은 18-19시간대에 1564 명의 사람들이 있을 예정입니다. 


In [ ]:
print(station['혜화'])

99


In [34]:
for i in range(7): # 혜화역 4호선
  temp = mean + b_st[99] + b_su[4] + b_we[i]
  result = int(np.exp(temp.item()) - 1)
  print(f"{days[i]} 혜화역 4호선은 18-19시간대에 {result} 명의 사람들이 있을 예정입니다. ")

월요일 혜화역 4호선은 18-19시간대에 6757 명의 사람들이 있을 예정입니다. 
화요일 혜화역 4호선은 18-19시간대에 7348 명의 사람들이 있을 예정입니다. 
수요일 혜화역 4호선은 18-19시간대에 7331 명의 사람들이 있을 예정입니다. 
목요일 혜화역 4호선은 18-19시간대에 7512 명의 사람들이 있을 예정입니다. 
금요일 혜화역 4호선은 18-19시간대에 7517 명의 사람들이 있을 예정입니다. 
토요일 혜화역 4호선은 18-19시간대에 3028 명의 사람들이 있을 예정입니다. 
일요일 혜화역 4호선은 18-19시간대에 2273 명의 사람들이 있을 예정입니다. 


In [ ]:
print(station['건대입구'])

20


In [35]:
for i in range(7): # 혜화역 4호선
  temp = mean + b_st[20] + b_su[7] + b_we[i]
  result = int(np.exp(temp.item()) - 1)
  print(f"{days[i]} 건대입구 7호선은 18-19시간대에 {result} 명의 사람들이 있을 예정입니다. ")

월요일 건대입구 7호선은 18-19시간대에 3037 명의 사람들이 있을 예정입니다. 
화요일 건대입구 7호선은 18-19시간대에 3302 명의 사람들이 있을 예정입니다. 
수요일 건대입구 7호선은 18-19시간대에 3295 명의 사람들이 있을 예정입니다. 
목요일 건대입구 7호선은 18-19시간대에 3376 명의 사람들이 있을 예정입니다. 
금요일 건대입구 7호선은 18-19시간대에 3378 명의 사람들이 있을 예정입니다. 
토요일 건대입구 7호선은 18-19시간대에 1360 명의 사람들이 있을 예정입니다. 
일요일 건대입구 7호선은 18-19시간대에 1021 명의 사람들이 있을 예정입니다. 
